In [3]:
import sys
import os
import time
import requests
import re
import numpy as np
import pandas as pd
import glob
from bs4 import BeautifulSoup

# --- VISUALIZATION ---
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns  

# --- MAPPING ---
import folium
from folium.plugins import HeatMap

# --- SELENIUM ---
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager

# --- GUI (TKINTER) ---
from tkinter import *
from tkinter import ttk, messagebox  # <--- THÊM messagebox VÀO ĐÂY
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import webbrowser

# --- CONFIG ---
HEADERS = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                         "AppleWebKit/537.36 (KHTML, like Gecko) "
                         "Chrome/128.0.0.0 Safari/537.36"}

In [4]:
# Cell này lấy danh sách các thành phố
OUTPUT_FILE = os.path.join('', 'cities_mapping.csv')
def get_cities_list():
    response = requests.get('https://insideairbnb.com/explore/', headers=HEADERS, timeout=10)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    data_list = []
    continents = soup.find_all('div', class_='continentContainer')
    

    for continent in continents:
        current_country = "Unknown"
        
        # Duyệt qua tất cả các thẻ <p> bên trong (vì Country và City đều là thẻ p nằm ngang hàng nhau)
        for p in continent.find_all('p'):
            
            # TRƯỜNG HỢP 1: Là tên Quốc gia (class="countryLabel")
            if 'countryLabel' in p.get('class', []):
                current_country = p.get_text(strip=True)
                
            # TRƯỜNG HỢP 2: Là tên Thành phố (class="cityLabel")
            elif 'cityLabel' in p.get('class', []):
                link_tag = p.find('a')
                if link_tag:
                    # Lấy tên thành phố (VD: Montreal)
                    city_name = link_tag.get_text(strip=True)
                    
                    # Lấy slug từ href (VD: /montreal/ -> montreal)
                    slug = link_tag['href'].strip('/')
                    
                    # Lấy vùng miền (VD: Quebec) - Phần chữ nằm ngoài thẻ <a>
                    # p.get_text() sẽ lấy cả "Montreal Quebec", ta xoá phần "Montreal" đi
                    full_text = p.get_text(" ", strip=True) 
                    region = full_text.replace(city_name, "").strip()
                    
                    # Tạo tên hiển thị đẹp cho Search Bar
                    # Ví dụ: "Montreal - Quebec (Canada)"
                    if region:
                        display_name = f"{city_name}, {region} ({current_country})"
                    else:
                        display_name = f"{city_name} ({current_country})"
                    
                    # Thêm vào danh sách
                    data_list.append({
                        'display_name': display_name, # Dùng để hiện trên Search bar
                        'slug': slug,                 # Dùng để ghép URL cào dữ liệu
                        'city': city_name,
                        'region': region,
                        'country': current_country
                    })

    # 3. Lưu ra CSV
    if data_list:
        df = pd.DataFrame(data_list)
        df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8')
        
        
        # Hiển thị 5 dòng đầu xem thử
        display(df.head())
    else:
        print("Không tìm thấy thành phố nào. Hãy kiểm tra lại cấu trúc web.")


In [5]:
#Cell này tải dữ liệu thành phố vào raw
OUTPUT_ROOT = "../../raw"
TARGET_FILES_ENDINGS = ["listings.csv.gz", "calendar.csv.gz", "reviews.csv.gz", "neighbourhoods.csv"]

def download_file(url, folder_path):
    filename = url.split("/")[-1]
    save_path = os.path.join(folder_path, filename)
    
    try:
        response = requests.get(url, headers=HEADERS)  
        response.raise_for_status()
    
        with open(save_path, 'wb') as f:
            f.write(response.content) 
    except Exception as e:
        print(f" Error: {e}")

def scrape_robust(city_name):
    
    options = Options() # Không để được headless cho chuyên nghiệp
    driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options=options)
    try:
        url = "http://insideairbnb.com/get-the-data/"
        driver.get(url)
        #Chuẩn hóa tên thành phố
        city_xpath = f"//h3[contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), '{city_name.lower()}')]"
        
        try:
            # 1. Tìm title Thành Phố
            city_header = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, city_xpath))
            )
            
            # Lướt đến thành phố cần thiết
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", city_header)
            time.sleep(1)

            # 2. Tìm nút Show nằm ngay sau tiêu đề đó
            # XPath: Từ thẻ H3 city, tìm thẻ 'a' kế tiếp có chứa chữ 'show'
            show_btn_xpath = f"{city_xpath}/following::a[contains(text(), 'show')][1]"
            
            show_button = driver.find_element(By.XPATH, show_btn_xpath)

            driver.execute_script("arguments[0].click();", show_button)
            time.sleep(5)
            
        except Exception as e:
            print(f"Lỗi không liên quan đến xử lý nút show: {e}")

        # 3. Tải dữ liệu
        soup = BeautifulSoup(driver.page_source, 'html.parser')


        all_h3 = soup.find_all('h3')
        target_h3 = None
        for h3 in all_h3:
            if city_name.lower() in " ".join(h3.get_text().split()).lower():
                target_h3 = h3
                break
        
        if not target_h3:
            print("Vẫn không thấy H3 trong HTML sau khi tải.")
            return

        current_date_str = None
        processed_count = 0
        found_snapshots = set()
        
        for element in target_h3.next_elements:
            # Gặp thành phố khác thì dừng
            if element.name == 'h3': break
            # Ngày tháng của snapshot đang xử lý
            if element.name == 'h4':
                raw_date = element.get_text(strip=True)
                match = re.search(r"(\d{1,2}\s+[A-Za-z]+\,?\s+\d{4})", raw_date)
                if match: current_date_str = match.group(1)
            
            if element.name == 'table':
                if not current_date_str: continue

                
                folder_path = os.path.join(OUTPUT_ROOT, city_name.lower(), current_date_str)
                
                links = element.find_all('a', href=True)
                folder_created = False
                # Tải
                for link in links:
                    f_url = link['href']
                    if os.path.basename(f_url) in TARGET_FILES_ENDINGS:
                        if not folder_created:
                            if current_date_str not in found_snapshots:
                                print(f"  Snapshot: {current_date_str}")
                                found_snapshots.add(current_date_str)
                            if not os.path.exists(folder_path): os.makedirs(folder_path)
                            folder_created = True
                        
                        download_file(f_url, folder_path)
                        processed_count += 1


    except Exception as e:
        print(f"Lỗi tổng: {e}")
    finally:
        driver.quit()
        pass
def crawl_longtitude_and_latitude(city_name):

    url = "https://nominatim.openstreetmap.org/search"

    
    params = {
        "q": city_name,
        "format": "json",
        "limit": 1
    }

    response = requests.get(url, params=params, headers=HEADERS)
    response.raise_for_status()
    
    data = response.json()
    
    if not data:
        raise ValueError(f"Không tìm thấy tọa độ cho thành phố: {city_name}")
    bbox = data[0].get("boundingbox")
    
    if not bbox:
         raise ValueError("Không tìm thấy boundingbox trong dữ liệu trả về")

    return {
        "min_latitude":  round(float(bbox[0]), 2),
        "max_latitude":  round(float(bbox[1]), 2),
        "min_longitude":  round(float(bbox[2]), 2),
        "max_longitude":  round(float(bbox[3]), 2)
    }


In [6]:
def clean_price(price_value):
    """Hàm làm sạch giá tiền (Giữ nguyên logic cũ)"""
    if pd.isna(price_value): return np.nan
    if isinstance(price_value, (int, float)): return float(price_value)
    s = str(price_value).strip()
    match = re.search(r"[-+]?[0-9,.]+", s.replace('$', ''))
    if not match: return np.nan
    num = match.group(0).replace(',', '') 
    try: return float(num)
    except: return np.nan


def process_city_data(city_name):
    """
    Hàm lọc và xử lý dữ liệu cho một thành phố cụ thể.
    
    Args:
        city_name (str): Tên folder thành phố (vd: 'brussels', 'berlin')
        lat_min, lat_max, lon_min, lon_max (float): Giới hạn toạ độ (Bounding Box)
        base_dir (str): Đường dẫn gốc dự án (mặc định là '..')
    """
    
    # Cấu hình đường dẫn động theo city_name
    RAW_DIR = os.path.join("../..", 'raw', city_name)
    PROCESSED_DIR = os.path.join("../..", 'processed', city_name)
    REPORTS_DIR = os.path.join("../..", 'reports')
    coors = crawl_longtitude_and_latitude(city_name)
    os.makedirs(PROCESSED_DIR, exist_ok=True)
    os.makedirs(REPORTS_DIR, exist_ok=True)


    snapshot_folders = [f for f in glob.glob(os.path.join(RAW_DIR, '*')) if os.path.isdir(f)]
    qa_summary_list = []

    print(f"--- BẮT ĐẦU XỬ LÝ {len(snapshot_folders)} SNAPSHOTS CHO: {city_name.upper()} ---")

    for folder_path in snapshot_folders:
        snapshot_name = os.path.basename(folder_path)
        print(f"\n>> Đang xử lý: {snapshot_name}")

        try:

            listings_df = pd.read_csv(os.path.join(folder_path, 'listings.csv.gz'), low_memory=True)
            calendar_df = pd.read_csv(os.path.join(folder_path, 'calendar.csv.gz'), low_memory=True)
            reviews_df = pd.read_csv(os.path.join(folder_path, 'reviews.csv.gz'), low_memory=False)
            neigh_df = pd.read_csv(os.path.join(folder_path, 'neighbourhoods.csv'))
        except FileNotFoundError:
            print(f"   [!] Thiếu file trong folder {snapshot_name}, bỏ qua.")
            continue

        # ---------------------------------------------------------
        # 1. XỬ LÝ LISTINGS (Cơ bản & Toạ độ)
        # ---------------------------------------------------------
        listings_df['price_numeric'] = listings_df['price'].apply(clean_price)
        
        # [QA1] Flag Price <= 0
        listings_df['qa_flag_price_zero'] = listings_df['price_numeric'].fillna(0) <= 0
        qa_summary_list.append({
            'snapshot_date': snapshot_name, 'rule_id': 'QA001_price_zero',
            'records_affected': int(listings_df['qa_flag_price_zero'].sum()),
            'handling_decision': 'Gắn cờ'
        })

        # Datetime & Coords
        listings_df['host_since'] = pd.to_datetime(listings_df['host_since'], errors='coerce')
        listings_df['latitude'] = pd.to_numeric(listings_df['latitude'], errors='coerce')
        listings_df['longitude'] = pd.to_numeric(listings_df['longitude'], errors='coerce')

        # [QA2] Check Coordinates Out of Bounds (Dùng tham số đầu vào)
        listings_df['qa_flag_out_of_city'] = (
            (listings_df['latitude'] < coors["min_latitude"]) | (listings_df['latitude'] > coors['max_latitude']) |
            (listings_df['longitude'] < coors['min_longitude']) | (listings_df['longitude'] > coors['max_longitude'])
        )
        qa_summary_list.append({
            'snapshot_date': snapshot_name, 'rule_id': 'QA002_coords_out_of_bounds',
            'records_affected': int(listings_df['qa_flag_out_of_city'].sum()),
            'handling_decision': 'Gắn cờ'
        })

        # [QA3] Duplicate IDs
        dups = listings_df.duplicated(subset=['id']).sum()
        if dups > 0:
            listings_df = listings_df.drop_duplicates(subset=['id'], keep='first')
        qa_summary_list.append({
            'snapshot_date': snapshot_name, 'rule_id': 'QA003_duplicate_ids',
            'records_affected': int(dups),
            'handling_decision': 'Xoá dòng trùng'
        })

        # ---------------------------------------------------------
        # 2. XỬ LÝ NEIGHBOURHOOD
        # ---------------------------------------------------------
        if 'neighbourhood_group' in neigh_df.columns: 
            neigh_df.drop(columns=['neighbourhood_group'], inplace=True)
        
        valid_neighbourhoods = set(neigh_df['neighbourhood'])
        
        # [QA4] Check valid neighbourhood
        col_neigh = 'neighbourhood_cleansed' if 'neighbourhood_cleansed' in listings_df.columns else 'neighbourhood'
        listings_df['qa_flag_invalid_neigh'] = ~listings_df[col_neigh].isin(valid_neighbourhoods)
        
        qa_summary_list.append({
            'snapshot_date': snapshot_name, 'rule_id': 'QA004_invalid_neighbourhood',
            'records_affected': int(listings_df['qa_flag_invalid_neigh'].sum()),
            'handling_decision': 'Gắn cờ'
        })

        # ---------------------------------------------------------
        # 3. XỬ LÝ REVIEWS
        # ---------------------------------------------------------
        reviews_df['date'] = pd.to_datetime(reviews_df['date'], errors='coerce')
        
        initial_reviews = len(reviews_df)
        reviews_df = reviews_df.dropna(subset=['comments'])
        

        valid_ids = set(listings_df['id'])
        reviews_df = reviews_df[reviews_df['listing_id'].isin(valid_ids)]
        
        removed_reviews = initial_reviews - len(reviews_df)
        qa_summary_list.append({
            'snapshot_date': snapshot_name, 'rule_id': 'QA005_orphaned_or_empty_reviews',
            'records_affected': int(removed_reviews),
            'handling_decision': 'Xoá bỏ'
        })

        # ---------------------------------------------------------
        # 4. XỬ LÝ CALENDAR
        # ---------------------------------------------------------
        calendar_df['date'] = pd.to_datetime(calendar_df['date'], errors='coerce')
        calendar_df['price_numeric'] = calendar_df['price'].apply(clean_price)
        if 'adjusted_price' in calendar_df.columns: calendar_df.drop(columns=['adjusted_price'], inplace=True)
        
        calendar_df = calendar_df[calendar_df['listing_id'].isin(valid_ids)]

        # ---------------------------------------------------------
        # 5. LƯU FILE
        # ---------------------------------------------------------
        out_dir = os.path.join(PROCESSED_DIR, snapshot_name)
        os.makedirs(out_dir, exist_ok=True)
        
        listings_df.to_csv(os.path.join(out_dir, 'listings_processed.csv'), index=False)
        calendar_df.to_csv(os.path.join(out_dir, 'calendar_processed.csv'), index=False)
        reviews_df.to_csv(os.path.join(out_dir, 'reviews_processed.csv'), index=False)
        neigh_df.to_csv(os.path.join(out_dir, 'neighbourhoods_processed.csv'), index=False)
        
        print(f"   -> Đã lưu xong: {out_dir}")

    # Lưu báo cáo QA tổng hợp cho thành phố đó
    report_file = os.path.join(REPORTS_DIR, f'qa_summary_{city_name}.csv')
    pd.DataFrame(qa_summary_list).to_csv(report_file, index=False)
    print(f"\n--- HOÀN TẤT XỬ LÝ {city_name.upper()}. Report saved to {report_file} ---")


In [7]:
def get_latest_processed_data(city_name):
    # Lưu ý: Cần đảm bảo biến PROCESSED_DIR trỏ đúng (dùng đường dẫn tương đối ../..)
    processed_root = os.path.join("../..", 'processed', city_name)
    
    if not os.path.exists(processed_root):
        print(f"Không tìm thấy dữ liệu cho {city_name} tại {processed_root}")
        return None
    
    subfolders = [f.path for f in os.scandir(processed_root) if f.is_dir()]
    if not subfolders: 
        print(f" Folder thành phố rỗng: {city_name}")
        return None
    
    latest_folder = max(subfolders, key=os.path.getctime)
    file_path = os.path.join(latest_folder, 'listings_processed.csv')
    
    if os.path.exists(file_path):
        return pd.read_csv(file_path)
    return None

def visualize_map(city_name):
    print(f"🗺️ ĐANG TẠO BẢN ĐỒ KHÔNG GIAN CHO: {city_name.upper()}")

    df = get_latest_processed_data(city_name)
    if df is None:
        print(" Không có dữ liệu.")
        return

    # --- LÀM SẠCH ---
    # Đảm bảo cột giá trị số tồn tại
    if 'price_numeric' not in df.columns:
        print(" Lỗi: Không tìm thấy cột 'price_numeric'. Hãy kiểm tra lại bước xử lý dữ liệu.")
        return

    df = df.dropna(subset=["latitude", "longitude", "price_numeric"])
    q_high = df["price_numeric"].quantile(0.99) # Lọc bỏ top 1% giá ảo
    df_clean = df[df["price_numeric"] < q_high].copy()

    # --- TÂM BẢN ĐỒ ---
    center_lat = df_clean["latitude"].mean()
    center_lon = df_clean["longitude"].mean()

    # --- TẠO BẢN ĐỒ ---
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=12,
        tiles="CartoDB positron"  
    )

    # ======================================================
    # 1️⃣ HEATMAP – MẬT ĐỘ LISTING (ĂN ĐIỂM PHÂN TÍCH KHÔNG GIAN)
    # ======================================================
    # Chuyển dữ liệu về dạng list các list toạ độ [Lat, Lon]
    heat_data = df_clean[["latitude", "longitude"]].values.tolist()
    
    HeatMap(
        data=heat_data,
        radius=10,
        blur=15,
        min_opacity=0.4,
        gradient={0.4: 'blue', 0.65: 'lime', 1: 'red'} # Màu nhiệt: Xanh -> Đỏ
    ).add_to(m)

    # ======================================================
    # 2️⃣ CIRCLE MARKER – GIÁ (AIRBNB STYLE)
    # ======================================================
    # Chỉ lấy mẫu 1000 điểm để trình duyệt không bị lag
    df_sample = df_clean.sample(min(1000, len(df_clean)), random_state=42)

    for _, row in df_sample.iterrows():
        price = row["price_numeric"]

        # Logic màu sắc
        if price < 80:
            color = "green" # Xanh lá (Rẻ)
        elif price < 150:
            color = "yellow" # Cam (Trung bình)
        else:
            color = "red" # Đỏ (Đắt)

        folium.CircleMarker(
            location=[row["latitude"], row["longitude"]],
            radius=5,
            fill=True,
            fill_opacity=0.7,
            fill_color=color,
            color="white", # Viền trắng cho nổi
            weight=1,
            tooltip=f"<b>{row.get('name', 'Phòng')}</b><br>Giá: ${int(price)}"
        ).add_to(m)

    # --- LƯU FILE HTML ---
    output_file = f"spatial_analysis_{city_name}.html"
    m.save(output_file)

    print(f" Đã tạo bản đồ thành công: {output_file}")
    
    # Hiển thị bản đồ ngay trong Notebook
    return m



In [8]:
def plot_room_type(city_name):
    # 1. Load dữ liệu (như cũ)
    df = get_latest_processed_data(city_name)
    if df is None: return

    # 2. Tính toán số liệu

    room_counts = df['room_type'].value_counts()
    total = len(df)
    
    # Thứ tự chuẩn

    categories = room_counts.index.tolist() 
    values = room_counts.values.tolist()
    percentages = [v/total * 100 for v in values]

    # 3. Cấu hình màu sắc 
    colors = []
    for cat in categories:
        if 'Entire home' in cat: colors.append('red') 
        elif 'Private room' in cat: colors.append('green') 
        else: colors.append('#767676')
    fig, ax = plt.subplots(figsize=(11, 7))

    fig.patch.set_facecolor('white') 
    ax.set_facecolor('white')

    # Vẽ thanh ngang
    y_pos = np.arange(len(categories))
    bars = ax.barh(y_pos, values, color=colors, height=0.7)
    
    # Đảo ngược trục Y
    ax.invert_yaxis() 
    


    ax.set_yticks(y_pos)
    ax.set_yticklabels(categories, fontsize=11, color='black', fontweight='bold')
    ax.tick_params(axis='x', colors='gray')
    ax.tick_params(axis='y', length=0)
    max_val = max(values)
    
    for i, (val, pct) in enumerate(zip(values, percentages)):
        if i == 0:
            ax.text(max_val *0.97, i - 0.9, "PHỔ BIẾN NHẤT", 
                    fontsize=10, fontweight='bold', color='black', ha='left')
            ax.text(max_val * 0.87, i - 0.65, f"{pct:.1f}%", 
                    fontsize=24, fontweight='bold', color='black', ha='left')
            ax.text(max_val * 1.04, i - 0.7, f"({categories[0]})", 
                    fontsize=12, fontweight='bold', color='black', ha='left')

        
        label_text = f"{val:,} ({pct:.1f}%)"

        
        ax.text(max_val * 1.05, i , label_text, 
                fontsize=12, fontweight='bold', color=colors[i], ha='left', va='center')
        
        ax.text(max_val * 1.05, i  +0.2, categories[i].lower(), 
                fontsize=10, color="#060505", ha='left')


    plt.title(f"Room Type - {city_name.upper()}", 
              fontsize=16, fontweight='bold', color='gray', loc='center', pad=20)
    
    plt.xlabel('Listings', fontsize=12)
    
    plt.tight_layout()
    plt.close(fig)
    return fig


In [9]:
def plot_activity(city_name):
    # Load dữ liệu
    df = get_latest_processed_data(city_name)
    if df is None: return
    

    # Giả định: Chỉ 30.5% khách viết review theo https://insideairbnb.com/data-assumptions/ với cả 30.5% cho dữ liệu khá giống cái biểu đồ trên explore the data
    REVIEW_RATE = 0.305
    # Xử lý dữ liệu
    df['number_of_reviews_ltm'] = df['number_of_reviews_ltm'].fillna(0)
    
    # (Inside Airbnb giả định booking trung bình tối thiểu là 3 đêm)
    adj_min_nights = df['minimum_nights'].apply(lambda x: max(min(x, 30), 3))
    
    # CÔNG THỨC: (Review 12 tháng / 0.5) * Min_Nights
    df['est_nights_booked'] = (df['number_of_reviews_ltm']/ REVIEW_RATE) * adj_min_nights
    # CAPPING: Giới hạn tối đa 255 đêm/năm (chiếm khoảng 70% năm)
    df['est_nights_booked'] = df['est_nights_booked'].clip(upper=255)
    # Tính thu nhập ước tính
    df_price_filtered = df[df['price_numeric'] <= 500]  
    df_price_filtered['est_income'] = (df_price_filtered['est_nights_booked']* df_price_filtered['price_numeric'])


    # CHIA NHÓM
    # Các mốc bắt trước explore the data : 0, 1-30... đến 241-255+
    bins = [-1, 0, 30, 60, 90, 120, 150, 180, 210, 240, 999]
    labels = ['0', '1-30', '31-60', '61-90', '91-120', '121-150', '151-180', '181-210', '211-240', '241-255+']
    
    df['occupancy_group'] = pd.cut(df['est_nights_booked'], bins=bins, labels=labels)
    group_counts = df['occupancy_group'].value_counts().reindex(labels) 
    avg_nights = df['est_nights_booked'].mean()
    avg_price = df_price_filtered['price_numeric'].mean()
    avg_income = df_price_filtered['est_income'].mean()

    # VẼ BIỂU ĐỒ
    fig, ax = plt.subplots(figsize=(11, 7))
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')


    x_pos = np.arange(len(labels))
    bars = ax.bar(x_pos, group_counts, color='#4682B4', width=0.85)

    # Trang trí trục và khung
    ax.set_xticks(x_pos)
    ax.set_xticklabels(labels, fontsize=9, color='#333333')
    
    # Ẩn khung viền trên và phải
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    
    # Label trục
    ax.set_xlabel('Occupancy (last 12 months)', fontsize=10, fontweight='bold', color='#333333')
    ax.set_ylabel('Listings', fontsize=12, fontweight='bold', color='#333333')

    
    # Dòng 1: Average Nights Booked
    plt.figtext(0.9, 0.78, f"{int(avg_nights)}", fontsize=32, fontweight='bold', color='#333333', ha='right')
    plt.figtext(0.9, 0.74, "average nights booked", fontsize=11, color='#666666', ha='right')
    
    # Dòng 2: Price/Night
    plt.figtext(0.9, 0.60, f"${int(avg_price)}", fontsize=22, fontweight='bold', color='#333333', ha='right')
    plt.figtext(0.9, 0.57, "price/night", fontsize=11, color='#666666', ha='right')
    
    # Dòng 3: Average Income
    plt.figtext(0.9, 0.42, f"€{int(avg_income):,}", fontsize=22, fontweight='bold', color='#333333', ha='right')
    plt.figtext(0.9, 0.39, "average income", fontsize=11, color='#666666', ha='right')

    # Tiêu đề
    plt.title(f"Activity - {city_name.upper()}", 
              fontsize=16, fontweight='bold', color='#004e66', loc='left', pad=20)
    
    plt.subplots_adjust(right=0.8) 
    plt.close(fig)
    return fig



In [10]:
def plot_license_analysis(city_name):
    # 1. Load dữ liệu
    df = get_latest_processed_data(city_name)
    if df is None: return

    # 2. XỬ LÝ DỮ LIỆU GIẤY PHÉP
    df['license'] = df['license'].astype(str).str.lower().str.strip()
    
    def categorize_license(val):
        if val == 'nan' or val == '' or val == 'none' or 'unlicensed' in val:
            return 'Unlicensed'
        elif 'exempt' in val:
            return 'Exempt'
        elif 'pending' in val:
            return 'Pending'
        else:
            return 'Licensed'

    df['license_status'] = df['license'].apply(categorize_license)
    
    # --- THAY ĐỔI 1: Đảm bảo luôn có đủ 4 nhóm (kể cả = 0) ---
    order = ['Licensed', 'Unlicensed', 'Exempt', 'Pending']
    
    # Dùng reindex để ép buộc phải có đủ các index trong 'order', thiếu thì điền 0
    status_counts = df['license_status'].value_counts().reindex(order, fill_value=0)
    
    labels = status_counts.index.tolist()
    counts = status_counts.values.tolist()
    total = sum(counts)
    

    max_idx = np.argmax(counts)
    winner_label = labels[max_idx]
    winner_count = counts[max_idx]
    winner_pct = (winner_count / total * 100) if total > 0 else 0

    # 3. VẼ BIỂU ĐỒ DONUT
    fig, ax = plt.subplots(figsize=(11, 7))
    fig.patch.set_facecolor('white')
    
    color_map = {
        'Licensed': '#1f77b4',   
        'Unlicensed': '#aec7e8', 
        'Exempt': '#ff7f0e',     
        'Pending': '#ffbb78'     
    }
    colors = [color_map[L] for L in labels]

    wedges, texts = ax.pie(
        counts, 
        startangle=90, 
        colors=colors,
        wedgeprops=dict(width=0.35, edgecolor='white')
    )
    

    unlicensed_count = status_counts.get('Unlicensed', 0)
    unlicensed_pct = (unlicensed_count / total) * 100
    # % UNLICENSED ---
    ax.text(1.3, 0.8, f"{unlicensed_pct:.1f}%", fontsize=36, fontweight='bold', color='#333333', ha='center')
    ax.text(1.3, 0.65, "unlicensed", fontsize=12, color='#666666', ha='center')
    # --- DANH SÁCH CHI TIẾT (Vòng lặp hiển thị cả số 0) ---
    y_start = 0.45
    for i, label in enumerate(labels):
        count = counts[i]
        pct = (count / total * 100) if total > 0 else 0
        color = color_map[label]

        text_stats = f"{count:,} ({pct:.1f}%)"
        

        weight = 'bold' if label == winner_label else 'normal'
        
        ax.text(1.3, y_start, text_stats, fontsize=12, fontweight='bold', color='#333333', ha='center')

        ax.text(1.3, y_start - 0.1, label.lower(), fontsize=11, color=color, ha='center', fontweight=weight)
        
        y_start -= 0.25 

    plt.title(f"Licenses Compliance - {city_name.upper()}", 
              fontsize=16, fontweight='bold', color='#004e66', loc='center', pad=20)
    

    
    # Tạo các "miếng giả" (proxy artists) để đảm bảo Legend luôn hiện đủ 4 màu

    patches = [mpatches.Patch(color=color_map[L], label=L) for L in labels]
    
    ax.legend(handles=patches, title="Status", loc="center", bbox_to_anchor=(0.5, 0.5), frameon=False)

    plt.tight_layout()
    plt.close(fig)
    return fig



In [11]:
def plot_short_term_rentals(city_name):
    #  Load dữ liệu
    df = get_latest_processed_data(city_name)
    if df is None: return

    # Chuyển đổi minimum_nights sang số
    df['minimum_nights'] = pd.to_numeric(df['minimum_nights'], errors='coerce')
    df = df.dropna(subset=['minimum_nights'])
    
    # Theo luật quốc tế chung, dưới 30 đêm là Short-term
    STR_THRESHOLD = 30
    
    # Phân loại
    short_term = df[df['minimum_nights'] < STR_THRESHOLD]
    long_term = df[df['minimum_nights'] >= STR_THRESHOLD]
    
    count_str = len(short_term)
    count_ltr = len(long_term)
    total = count_str + count_ltr
    
    pct_str = (count_str / total) * 100
    pct_ltr = (count_ltr / total) * 100


    plot_data = df['minimum_nights'].clip(upper=35)
    

    counts = plot_data.value_counts().sort_index()
    
    #  VẼ BIỂU ĐỒ
    fig, ax = plt.subplots(figsize=(11, 7))
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')
    

    bars = ax.bar(counts.index, counts.values, color='#1f77b4', width=0.6)
    

    # Vẽ đường đứt nét tại mốc 30 đêm
    ax.axvline(x=STR_THRESHOLD - 0.5, color='#333333', linestyle='--', linewidth=2, alpha=0.7)
    
    # Chú thích cho đường ranh giới (Xoay dọc 90 độ)
    ax.text(STR_THRESHOLD - 1.5, ax.get_ylim()[1]*0.5, 'STR Threshold (30 days)', 
            rotation=90, color='#333333', fontweight='bold', ha='center')

    #  TRANG TRÍ TRỤC
    # Chỉ hiển thị các mốc quan trọng ở trục X giống ảnh mẫu
    major_ticks = [1, 2, 3, 4, 5, 6, 7, 14, 21, 28, 35]
    ax.set_xticks(major_ticks)
    
    # Đổi nhãn 35 thành "35+"
    xtick_labels = [str(t) if t < 36 -1 else "35+" for t in major_ticks]
    ax.set_xticklabels(xtick_labels, fontsize=10)
    
    ax.set_xlabel('Minimum Nights', fontsize=11, fontweight='bold', color='#333333')
    ax.set_ylabel('Listings', fontsize=11, fontweight='bold', color='#333333')
    
    # Ẩn khung viền (Despine)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_color('#888888')
    ax.spines['bottom'].set_color('#888888')

    # --- BIG NUMBER ---
    plt.figtext(0.75, 0.78, f"{pct_str:.1f}%", fontsize=36 - 9, fontweight='bold', color='#333333', ha='right')
    plt.figtext(0.75, 0.75, "short-term rentals", fontsize=36 - 26, color="#A61F1F", ha='right')
    

    # Dòng 1: Short-term
    plt.figtext(0.75, 0.60, f"{count_str:,} ({pct_str:.1f}%)", fontsize=10, fontweight='bold', color='#333333', ha='right')
    plt.figtext(0.75, 0.58, "short-term rentals", fontsize=9, color='#666666', ha='right')
    
    # Dòng 2: Long-term
    plt.figtext(0.75, 0.45, f"{count_ltr:,} ({pct_ltr:.1f}%)", fontsize=10, fontweight='bold', color='#333333', ha='right')
    plt.figtext(0.75, 0.43, "longer-term rentals", fontsize=9, color='#666666', ha='right')

    # Tiêu đề
    plt.title(f"Short-Term Rentals Analysis - {city_name.upper()}", 
              fontsize=16, fontweight='bold', color='#004e66', loc='center', pad=20)
    
    # Thu hẹp biểu đồ để nhường chỗ cho Text bên phải
    plt.subplots_adjust(right=0.75)
    plt.close(fig)
    return fig


In [12]:
def plot_listings_per_host(city_name):
    #  Load dữ liệu
    df = get_latest_processed_data(city_name)
    if df is None: return

    #  XỬ LÝ DỮ LIỆU
    # Đảm bảo cột calculated_host_listings_count là số
    df['calculated_host_listings_count'] = pd.to_numeric(df['calculated_host_listings_count'], errors='coerce')
    
    # Gom nhóm: Tất cả các host có > 10 phòng sẽ được gộp vào nhóm 10
    #  Trục X là "Listings per host", Trục Y là "Số lượng Listings" thuộc nhóm đó
    plot_data = df['calculated_host_listings_count'].clip(upper=10)
    
    # Đếm số lượng listing trong từng nhóm (1, 2, ..., 10+)
    counts = plot_data.value_counts().sort_index()
    

    # Single: Nhóm 1
    single_listings_count = counts.get(1, 0)
    
    # Multi: Tổng các nhóm từ 2 đến 10+
    multi_listings_count = len(df) - single_listings_count
    
    total_listings = single_listings_count + multi_listings_count
    
    # Tính phần trăm
    pct_multi = (multi_listings_count / total_listings) * 100 if total_listings > 0 else 0
    pct_single = (single_listings_count / total_listings) * 100 if total_listings > 0 else 0

    # 3. VẼ BIỂU ĐỒ
    fig, ax = plt.subplots(figsize=(11, 7))
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')
    
    # Vẽ các cột 
    x_labels = [str(int(i)) if i < 10 else "10+" for i in counts.index]
    
    bars = ax.bar(x_labels, counts.values, color='#4682B4', width=0.7) # SteelBlue
    
    # TRANG TRÍ TRỤC
    ax.set_xlabel('Listings per Host', fontsize=11, fontweight='bold', color='#333333')
    ax.set_ylabel('Listings', fontsize=11, fontweight='bold', color='#333333')
    
    # Ẩn khung viền
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_color('#888888')
    ax.spines['bottom'].set_color('#888888')
    
    # --- BIG NUMBER: % MULTI-LISTINGS ---
    plt.figtext(0.85, 0.78, f"{pct_multi:.1f}%", fontsize=36, fontweight='bold', color='#333333', ha='right')
    plt.figtext(0.85, 0.74, "multi-listings", fontsize=12, color='#666666', ha='right')
    
    # Dòng 1: Single Listings
    plt.figtext(0.85, 0.60, f"{single_listings_count:,} ({pct_single:.1f}%)", fontsize=14, fontweight='bold', color='#333333', ha='right')
    plt.figtext(0.85, 0.57, "single listings", fontsize=11, color='#666666', ha='right')
    
    # Dòng 2: Multi Listings
    plt.figtext(0.85, 0.45, f"{multi_listings_count:,} ({pct_multi:.1f}%)", fontsize=14, fontweight='bold', color='#333333', ha='right')
    plt.figtext(0.85, 0.42, "multi-listings", fontsize=11, color='#666666', ha='right')

    # Tiêu đề
    plt.title(f"Listings per Host - {city_name.upper()}", 
              fontsize=16, fontweight='bold', color='#004e66', loc='left', pad=20)
    
    # Thu hẹp biểu đồ để nhường chỗ cho Text bên phải
    plt.subplots_adjust(right=0.85)
    plt.close(fig)
    return fig



In [13]:
def plot_list_top_30_hosts(city_name):
    # LOAD VÀ XỬ LÝ DỮ LIỆU 
    df = get_latest_processed_data(city_name)
    if df is None: return None 

    df['host_name'] = df['host_name'].fillna('Unknown')
    
    host_view = df.pivot_table(
        index=['host_id', 'host_name'],
        columns='room_type',
        values='id',
        aggfunc='count',
        fill_value=0
    )

    cols = ['Entire home/apt', 'Private room', 'Shared room', 'Hotel room']
    host_view = host_view.reindex(columns=cols, fill_value=0)

    host_view['Total'] = host_view.sum(axis=1)
    
    # Lấy Top 30
    top_30 = host_view.sort_values(by='Total', ascending=False).head(30)

    display_df = top_30.reset_index().drop(columns=['host_id'])
    display_df['host_name'] = display_df['host_name'].apply(lambda x: str(x)[:25] + '..' if len(str(x)) > 25 else str(x))
    display_df.columns = ['Host Name', 'Entire', 'Private', 'Shared', 'Hotel', 'TOTAL']

    fig_height = len(display_df) * 0.3 + 1 
    fig, ax = plt.subplots(figsize=(10, fig_height)) 

    # Ẩn trục tọa độ (x, y)
    ax.axis('tight')
    ax.axis('off')
    table = ax.table(
        cellText=display_df.values,
        colLabels=display_df.columns,
        loc='center',
        cellLoc='center',
        colColours=["#61C416"]*len(display_df.columns) 
    )

    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.5) 
    for (row, col), cell in table.get_celld().items():
        if row == 0:
            cell.set_text_props(color='white', weight='bold')

    ax.set_title(f"Top 30 Hosts in {city_name}", fontsize=14, fontweight='bold', pad=20)

    plt.tight_layout()
    plt.close(fig)
    
    return fig 


In [14]:
class SearchableComboBox:
    def __init__(self, parent, options):
        self.parent = parent
        self.options = options
        self.dropdown_id = None

        # Tạo Frame gói bên ngoài (Wrapper)
        # Dùng Frame thường (không phải ttk) để dễ chỉnh màu nền trắng
        self.wrapper = Frame(parent, bg="white") 

        # Ô nhập liệu (Entry)
        self.entry = ttk.Entry(self.wrapper, font=("Segoe UI", 14))
        self.entry.bind("<KeyRelease>", self.on_entry_key) # Đã sửa lỗi chính tả KeyRealease
        self.entry.bind("<FocusIn>", self.show_dropdown)
        self.entry.pack(side=LEFT, fill=BOTH, expand=True)

        # Nút mũi tên (Thay icon bằng text "▼" để code chạy ngay không cần file ảnh)
        self.btn = Button(self.wrapper, text="▼", font=("Segoe UI", 10),
                          relief="flat", bg="white", command=self.toggle_dropdown)
        self.btn.pack(side=RIGHT, fill=Y)

        # Listbox (Danh sách gợi ý)
        # Lưu ý: Dùng Listbox của tkinter (ttk không có Listbox)
        self.listbox = Listbox(parent, font=("Segoe UI", 12), height=10, bg="white", bd=1, relief="solid")
        self.listbox.bind("<<ListboxSelect>>", self.on_select)
        
        self.update_listbox(self.options)

    # Hàm hỗ trợ để pack wrapper vào giao diện chính
    def pack(self, **kwargs):
        self.wrapper.pack(**kwargs)

    # Hàm lấy giá trị hiện tại
    def get(self):
        return self.entry.get()

    def update_listbox(self, items):
        self.listbox.delete(0, END)
        for item in items:
            self.listbox.insert(END, item)

    def on_entry_key(self, event):
        typed_value = event.widget.get().strip().lower()
        if not typed_value:
            filtered_options = self.options
        else:
            filtered_options = [opt for opt in self.options if opt.lower().startswith(typed_value)]
        
        self.update_listbox(filtered_options)
        self.show_dropdown()

    def on_select(self, event):
        if self.listbox.curselection():
            selected_index = self.listbox.curselection()[0]
            selected_option = self.listbox.get(selected_index)
            
            self.entry.delete(0, END)
            self.entry.insert(0, selected_option)
            self.hide_dropdown()

    def toggle_dropdown(self):
        if self.listbox.winfo_ismapped():
            self.hide_dropdown()
        else:
            self.show_dropdown()

    def show_dropdown(self, event=None):
        # Tính toán vị trí để Listbox hiện ngay dưới Entry
        x = self.wrapper.winfo_x()
        y = self.wrapper.winfo_y() + self.wrapper.winfo_height()
        w = self.wrapper.winfo_width()
        
        # Dùng place để đè lên các widget khác
        self.listbox.place(in_=self.wrapper, x=0, rely=1.0, relwidth=1.0, anchor="nw")
        self.listbox.lift()

        # Tự động ẩn sau 5 giây nếu không thao tác
        if self.dropdown_id:
            self.listbox.after_cancel(self.dropdown_id)
        self.dropdown_id = self.listbox.after(5000, self.hide_dropdown)

    def hide_dropdown(self, event=None):
        self.listbox.place_forget()


# --- 2. MAIN APP ---
# --- MAIN APP (GUI) ---
def Main_app():
    try:
        df = pd.read_csv('cities_mapping.csv')
        all_cities = sorted(df['city'].dropna().astype(str).unique().tolist())
    except: 
        all_cities = ["London", "Paris", "New York"]

    window = Tk()
    window.title("Data Explorer")
    window.state('zoomed') # Phóng to toàn màn hình
    window.config(background="#F7F9FC")

    current_frame = None

    def switch_frame(new_frame_func, *args):
        nonlocal current_frame
        if current_frame:
            current_frame.destroy()
        current_frame = new_frame_func(*args)
        current_frame.pack(fill="both", expand=True)

    # --- MÀN HÌNH TÌM KIẾM ---
    def create_search_screen():
        frame = Frame(window, bg="#F7F9FC")
        card = Frame(frame, bg="white", padx=40, pady=40)
        card.place(relx=0.5, rely=0.5, anchor="center")

        Label(card, text="Data Explorer", font=("Segoe UI", 30, "bold"), fg="#FF5A5F", bg="white").pack(pady=(0, 10))
        
        search_combo = SearchableComboBox(card, all_cities)
        search_combo.pack(ipady=5, pady=20, fill=X)

        def run_analysis():
            city = search_combo.get()
            if city:
                switch_frame(create_analysis_screen, city)
            else:
                from tkinter import messagebox
                messagebox.showwarning("Nhắc nhở", "Vui lòng chọn một thành phố!")

        Button(card, text="PHÂN TÍCH DỮ LIỆU", font=("Segoe UI", 12, "bold"), bg="#FF5A5F", fg="white",
               relief="flat", padx=30, pady=12, command=run_analysis).pack()
        return frame

    # --- MÀN HÌNH DASHBOARD ---
    def create_analysis_screen(city_name):
        frame = Frame(window, bg="white")
        
        # Header
        header = Frame(frame, bg="#FF5A5F", height=60)
        header.pack(fill=X, side=TOP)
        Button(header, text="⬅ Quay lại", font=("Segoe UI", 11), bg="white", fg="#FF5A5F", relief="flat",
               command=lambda: switch_frame(create_search_screen)).pack(side=LEFT, padx=20, pady=10)
        Label(header, text=f"BÁO CÁO PHÂN TÍCH: {city_name.upper()}", font=("Segoe UI", 16, "bold"), bg="#FF5A5F", fg="white").pack(side=LEFT, padx=20)

        # Body
        content = Frame(frame, bg="white")
        content.pack(fill=BOTH, expand=True)

        # Cột Trái: Thông tin & Map
        left_panel = Frame(content, bg="#F7F7F7", width=400, padx=20, pady=20)
        left_panel.pack(side=LEFT, fill=Y)
        left_panel.pack_propagate(False) 

        Label(left_panel, text="TIẾN TRÌNH", font=("Segoe UI", 14, "bold"), bg="#F7F7F7", fg="#333").pack(anchor="w")
        status_log = Text(left_panel, height=15, width=40, font=("Consolas", 9), bg="white", relief="flat", state=DISABLED)
        status_log.pack(pady=10, fill=X)

        Label(left_panel, text="BẢN ĐỒ ĐỊA LÝ", font=("Segoe UI", 14, "bold"), bg="#F7F7F7", fg="#333").pack(anchor="w", pady=(20, 0))
        btn_open_map = Button(left_panel, text="Mở Bản Đồ (phải lên web)", font=("Segoe UI", 11), bg="#00A699", fg="white", 
                              state=DISABLED, command=lambda: webbrowser.open(f"spatial_analysis_{city_name}.html"))
        btn_open_map.pack(pady=10, fill=X)

        # Cột Phải: Biểu đồ (Có thanh cuộn)
        right_panel = Frame(content, bg="white")
        right_panel.pack(side=RIGHT, fill=BOTH, expand=True)

        canvas = Canvas(right_panel, bg="white")
        scrollbar = Scrollbar(right_panel, orient="vertical", command=canvas.yview)
        scrollable_frame = Frame(canvas, bg="white")

        scrollable_frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        
        canvas.pack(side=LEFT, fill=BOTH, expand=True)
        scrollbar.pack(side=RIGHT, fill=Y)

        # Hàm ghi log
        def log(msg):
            status_log.config(state=NORMAL)
            status_log.insert(END, ">> " + msg + "\n")
            status_log.see(END)
            status_log.config(state=DISABLED)
            window.update()


        def run_pipeline():
            try:
                # B1: Cào dữ liệu
                log("Đang cào dữ liệu ")
                scrape_robust(city_name) 
                
                # B2: Xử lý
                log("Đang làm sạch dữ liệu")
                process_city_data(city_name)

                # B3: Map
                log("Đang tạo bản đồ")
                visualize_map(city_name) 
                
                # Kiểm tra file map có tồn tại không để bật nút
                if os.path.exists(f"spatial_analysis_{city_name}.html"):
                    btn_open_map.config(state=NORMAL, bg="#00A699")
                    log("Đã tạo xong bản đồ!")
                else:
                    log("Không tìm thấy file bản đồ (Code visualize_map chưa chạy?)")

                # B4: Vẽ biểu đồ
                log("Đang vẽ các biểu đồ thống kê...")
                
                charts = [
                    plot_room_type, 
                    plot_activity, 
                    plot_license_analysis, 
                    plot_short_term_rentals, 
                    plot_listings_per_host,
                    plot_list_top_30_hosts
                ]

                for chart_func in charts:
                    try:
                        log(f"Đang vẽ: {chart_func.__name__}")
                        window.update()
                        
                        fig = chart_func(city_name)
                        
                        if fig:
                            fig.set_size_inches(11, 7) 
                        
                            fig.tight_layout()

                            canvas_plot = FigureCanvasTkAgg(fig, master=scrollable_frame)
                            canvas_plot.draw()
                            widget = canvas_plot.get_tk_widget()
                            
                            # Vẫn dùng pack() để xếp hàng dọc
                            widget.pack(pady=10, padx=20, fill=X) 
                            
                        else:
                            log(f"Không có dữ liệu cho {chart_func.__name__}")
                    except Exception as e:
                        log(f"Lỗi vẽ {chart_func.__name__}: {e}")

                log("--- HOÀN TẤT ---")

            except Exception as e:
                log(f"LỖI NGHIÊM TRỌNG: {e}")

        # Chạy sau 100ms
        window.after(100, run_pipeline)
        return frame

    switch_frame(create_search_screen)
    window.mainloop()

Main_app()